In [2]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install pandas

     |████████████████████████████████| 285 kB 8.0 MB/s eta 0:00:01


In [3]:
# importeert belangrijke functionaliteit
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
import calendar

# importeert de excel gegevens
data = pd.read_excel (r'F:\gebruiker\Documents\COVID-19_casus_landelijk (1).xlsx')
h_d = pd.DataFrame(data, columns= ['Date_statistics', 'Hospital_admission', 'Deceased', 'Week_of_death', 'Agegroup']) # exl bestand met deze drie kolommen

# dit deel creeert een extra week col. met de corresponderende week van de gegeven dag
h_d['Date_statistics'] = h_d['Date_statistics'].apply(lambda x: pd.Timestamp(x).strftime('%d-%m-%Y'))
h_d['Date_statistics'] = pd.to_datetime(h_d['Date_statistics'], format='%d-%m-%Y')
h_d['Week_number'] = h_d['Date_statistics'].dt.isocalendar().week

#deaths = h_d[h_d['Deceased'] == 1]
#print(deaths.shape)
agedist_h_d = h_d.copy()
agedist_h_d.replace('2019-10-01 00:00:00', '10-19')
agedist_h_d.drop(agedist_h_d.index[agedist_h_d['Agegroup'] == 'Unknown'], inplace = True)
age_dist = pd.crosstab(agedist_h_d.Agegroup, agedist_h_d.Deceased)
age_dist.drop(age_dist.index[0])
print(age_dist)
ax = age_dist.plot.bar(stacked=True)
plt.legend(['overleeft','overleden','onbekend'],title='status')
plt.show()

# alle coronagevallen nederland
test = pd.crosstab(h_d.Date_statistics, h_d.Deceased)
ax = test.plot.bar(stacked=True)
plt.xticks(np.arange(0, 400, step=7))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m"))
plt.legend(['overleeft','overleden','onbekend'],title='status')
plt.xticks(np.arange(0, 320, step=31))
plt.show()

weeks = h_d.groupby('Date_statistics')['Date_statistics'].count()
days = h_d.groupby('Date_statistics')['Date_statistics'].count()
format = {'Amount_weeks': weeks, 'Cases': days}
cases = pd.DataFrame(format)
cases['day'] = cases.index
sns.lineplot(data=cases, x='day',y='Cases')
plt.show()

# een df zonder de gevallen zonder ziekenhuis opname
# h_d_2 = h_d[h_d['Hospital_admission'] != 0]

# alleen doden waar ook de week van dood bekend is
h_d_3 = h_d.dropna(axis=0, how='any')
h_d_3['Weeks_of_illness'] = h_d_3.apply(lambda x: x['Week_of_death'] - x['Week_number'], axis=1)
print(h_d_3.shape)
print(h_d_3['Weeks_of_illness'].mean())
print(h_d_3)

# tabel 1 doden tegenover ziekenhuis opname. tabel 2 week nummer tegenover besmettingenen met dood hue
tab = pd.crosstab(h_d['Deceased'], h_d['Hospital_admission'], margins=True)
print(tab)
tab = pd.crosstab(h_d['Week_number'], h_d['Deceased'], margins=True)
print(tab)


# wat plots
sns.countplot(x='Hospital_admission', hue='Deceased', data=h_d) # grafiek van alle mogelijkheden voor Hospital_admission en de corresponderende doden
plt.show()
#sns.countplot(x='Hospital_admission', hue='Deceased', data=h_d_2) # grafiek met alleen opnames en unknown en de corresponderende doden
#plt.show()
#sns.countplot(x='Week_number', hue='Deceased', data=h_d_2) # grafiek met de gevallen per week met het aantal doden weergegeven
plt.show()

# berekening gemiddelde duur hospitalisatie van corona-gevallen die zijn overleden
amount_weeks = h_d_3.groupby('Week_number')['Weeks_of_illness'].sum()
# print(amount_weeks)
days = h_d_3.groupby('Week_number')['Date_statistics'].count()
# print(days)
format = {'Amount_weeks': amount_weeks, 'Cases': days}
mean_weeks = pd.DataFrame(format)
mean_weeks['Mean_hospitalisation'] = mean_weeks['Amount_weeks'] / mean_weeks['Cases']
mean_weeks.reset_index(inplace=True)
mean_weeks = mean_weeks.rename(columns = {'index':'Week_number'})
print(mean_weeks)

# plot
sns.lineplot(data=mean_weeks, x="Week_number", y="Mean_hospitalisation")
plt.show()

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.